In [1]:
## initiate the open source model

from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model="llama2")
llm.invoke("what is the capital of west bengal?")

'The capital of West Bengal is Kolkata (formerly known as Calcutta).'

In [3]:
## loading the documents

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("offer.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'offer.pdf', 'page': 0}, page_content='exl Service.com (India) Private Limited \n1st Floor, Tower 1, NSL Techzone, Plot No. 8, Sector 144, NSEZ, Noida, UP – 201306 EXLservice.com \nRegistered Office: 414, 4th Floor, DLF Jasola Tower B, Plot No. 10 & 11, DDA District Centre, Jasola, New Delhi – 110 044, India. \n \n  \nPRIVATE AND CONFIDENTIAL \n \nDate: September 22, 2024         \n \nMr. Sayanik  Mukherjee     \nSammadar Para,Gobardanga,Habra \nWest Bengal-743252 \n \nEMPLOYMENT AGREEMENT \n \n \nDear Sayanik, \n \nFurther to the recent meetings and discussions you had with us, we are now pleased to offer you an \nappointment with exl Service.com (India) Private Limited  (“the Company”) to the position of \nReporting Analyst - Analytics at Band A1, on the terms and conditions set out herein after: \n \n1 EMPLOYMENT  \n \n1.1 Your effective date of joining shall be no later than : September 23, 2024 \n \n1.2 Notwithstanding the afore -said or anything to t

In [4]:
## creating prompt template

from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below.If you don't know the answer of the question please reply I don't know

Context:{context}

Question:{question}
"""

prompt = PromptTemplate.from_template(template)

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = prompt | llm | parser

In [6]:
chain.input_schema.schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [ ]:
from pydantic import model_validator
from langchain_ollama.embeddings import OllamaEmbeddings
embedding = OllamaEmbeddings(model="llama2") 

In [8]:
## storing embeddings

from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(pages,embedding)

In [9]:
retriever = vectorstore.as_retriever()

# want top 5 semantic context
retriever.search_kwargs = {"k": 5}

query = "compensation"
results = retriever.invoke(query)
results

[Document(metadata={'source': 'offer.pdf', 'page': 4}, page_content='on termination of employment (including from salary, salary in lieu of notice, holiday pay, \nallowances, discretionary bonus (if any), etc.) all debts owed by you to the Company or any of \nits group or affiliate companies or any fine or recovery imposed by the Company including \npursuant to the Company’s disciplinary procedure or to deduct any amount for absence from \nduty or for notice period not served by you or for damage to or loss of goods or Company assets \nor for recovery of advances or loans, etc.'),
 Document(metadata={'source': 'offer.pdf', 'page': 13}, page_content='the management to decide whether any such particular benefit will be provided to you \nor not. \n \n \nAllowances (as applicable) \nYou may get paid following allowances as per the following specifications and \nannualized limits.'),
 Document(metadata={'source': 'offer.pdf', 'page': 13}, page_content="exl Service.com (India) Private Limite

In [11]:
from operator import itemgetter
chain = (
    { "context": itemgetter("question") | retriever,"question":itemgetter("question")}
    | prompt
    | llm
    |parser)

chain.invoke({'question':"what is the exact position that was offererd to Sayanik Mukherjee by EXL services?"})

'The offered position to Sayanik Mukherjee by EXL Services is "Reporting Analyst - Analytics" at Band A1.'